In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [1]:
url = "https://www.flipkart.com/motorola-edge-50-fusion-marshmallow-blue-128-gb/p/itmf88eea5799a27?pid=MOBGXTYZEZSZQE7W&lid=LSTMOBGXTYZEZSZQE7WIBXLBI&marketplace=FLIPKART&store=tyy%2F4io&srno=b_1_1&otracker=browse&fm=organic&iid=6bf71f72-897f-4e5f-ac21-be2b66d7d754.MOBGXTYZEZSZQE7W.SEARCH&ppt=hp&ppn=homepage&ssid=wri8o089dc0000001743772848143"

In [3]:
# Headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

In [4]:
# Fetch HTML content
response = requests.get(url, headers=headers)

In [5]:
soup = BeautifulSoup(response.content, "html.parser")

In [6]:
# Extract Data
# 1. Product Name
product_name = soup.find("span", class_="VU-ZEz").text.strip()

In [7]:
# 2. Product price
header_section = soup.find("div", class_ = "C7fEHH")
price = header_section.find("div", class_ = "Nx9bqj CxhGGd").text.strip()

In [27]:
price

'₹22,999'

In [8]:
# Product Specifications
specs = {}
spec_section = soup.find("div", class_="_3Fm-hO")
if spec_section:
    tables = spec_section.find_all("tr", class_="WJdYP6 row")
    for row in tables:
        key = row.find("td", class_="+fFi1w col col-3-12").text.strip()
        value = row.find("td", class_="Izz52n col col-9-12").text.strip()
        specs[key] = value

In [9]:
specs

{'In The Box': 'Handset, 68W Charger, USB Cable, Sim Tool, Protective Cover, Guides',
 'Model Number': 'PB300002IN|PB300010IN',
 'Model Name': 'Edge 50 Fusion',
 'Color': 'Marshmallow Blue',
 'Browse Type': 'Smartphones',
 'SIM Type': 'Dual Sim',
 'Hybrid Sim Slot': 'No',
 'Touchscreen': 'Yes',
 'OTG Compatible': 'Yes',
 'Quick Charging': 'Yes',
 'Sound Enhancements': 'Stereo Speakers',
 'SAR Value': 'Head: 1.04 W/Kg (at 1 g), Body: 1.05 W/Kg (at 1 g)',
 'Display Size': '17.02 cm (6.7 inch)',
 'Resolution': '2400 x 1080 Pixels',
 'Resolution Type': 'Full HD+',
 'GPU': 'Adreno 710',
 'Display Type': 'Full HD+ pOLED Endless Edge Display',
 'Other Display Features': '144Hz Refresh Rate, 10 Bit, 100% DCI P3, 1600 Peak Nits, 1200 HBM Nits, 360Hz Game Mode, Aqua Touch, 720Hz PWM/DC Dimming, Corning Glass 5, SGS Low Blue Light, SGS Low Motion Blur, Aspect Ratio: 20:9, Screen-to-Body Ratio: 92%',
 'Operating System': 'Android 14',
 'Processor Brand': 'Snapdragon',
 'Processor Type': '7s Gen 2'

In [10]:
product_name

'Motorola Edge 50 Fusion (Marshmallow Blue, 128 GB)\xa0\xa0(8 GB RAM)'

In [11]:
price

'₹22,999'

In [12]:
documents = []
text = f"Poduct: {product_name}. Price: {price}. "
text += "Specifications: " + ". ".join([f"{key}: {value}" for key, value in specs.items()])
documents.append(text)

In [13]:
documents

["Poduct: Motorola Edge 50 Fusion (Marshmallow Blue, 128 GB)\xa0\xa0(8 GB RAM). Price: ₹22,999. Specifications: In The Box: Handset, 68W Charger, USB Cable, Sim Tool, Protective Cover, Guides. Model Number: PB300002IN|PB300010IN. Model Name: Edge 50 Fusion. Color: Marshmallow Blue. Browse Type: Smartphones. SIM Type: Dual Sim. Hybrid Sim Slot: No. Touchscreen: Yes. OTG Compatible: Yes. Quick Charging: Yes. Sound Enhancements: Stereo Speakers. SAR Value: Head: 1.04 W/Kg (at 1 g), Body: 1.05 W/Kg (at 1 g). Display Size: 17.02 cm (6.7 inch). Resolution: 2400 x 1080 Pixels. Resolution Type: Full HD+. GPU: Adreno 710. Display Type: Full HD+ pOLED Endless Edge Display. Other Display Features: 144Hz Refresh Rate, 10 Bit, 100% DCI P3, 1600 Peak Nits, 1200 HBM Nits, 360Hz Game Mode, Aqua Touch, 720Hz PWM/DC Dimming, Corning Glass 5, SGS Low Blue Light, SGS Low Motion Blur, Aspect Ratio: 20:9, Screen-to-Body Ratio: 92%. Operating System: Android 14. Processor Brand: Snapdragon. Processor Type: 7

In [14]:
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

In [15]:
# --- Generate Embeddings ---
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(documents)

In [16]:
# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))

In [21]:
# --- QA Pipeline ---
qa_pipeline = pipeline(
    "question-answering",
    model="bert-large-uncased-whole-word-masking-finetuned-squad"
)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [22]:
def answer_question(question):
    # Search for relevant documents
    query_embedding = model.encode([question])
    distances, indices = index.search(query_embedding.astype('float32'), k=3)
    
    # Get context from the most relevant document
    context = documents[indices[0][0]]
    
    # Get answer from BERT
    result = qa_pipeline(question=question, context=context)
    
    return result["answer"] if result["score"] > 0.1 else "I don't know."

In [23]:
# --- Test ---
print(answer_question("What is the price?"))       
print(answer_question("Does it have Bluetooth?"))   
print(answer_question("What's the battery capacity?"))  

₹22,999
Bluetooth Support: Yes. Bluetooth Version: v5.2
5000 mAh


In [25]:
print(answer_question("What is the color of product?"))

Blue


In [26]:
print(answer_question("What is the size"))

0.7um Pixel Size


In [24]:
import gradio as gr
gr.Interface(fn=answer_question, inputs="text", outputs="text").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
